In [ ]:
import pandas as pd
import numpy as np
import json
from ast import literal_eval
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
credits = pd.read_csv("/content/tmdb_5000_credits.csv", engine='python', on_bad_lines='skip')
movies = pd.read_csv("/content/tmdb_5000_movies.csv", engine='python', on_bad_lines='skip')

In [ ]:
movies.head()

budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   
3  [{"id": 849, "name": "dc comics"}, {"id": 853,...                en   
4  [{"id": 818, "name": "based on novel"}, {"id":...                en   

                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   
2  A cryptic message from Bond’s past sends him o...  107.376788   
3  Following the death of District Attorney Harve...  112.312950   
4  John Carter is a war-weary, former military ca...   43.926995   

                                production_companies  \
0  [{"name": "Ingenious Film Partners", "id": 289...   
1  [{"name": "Walt Disney Pictures", "id": 2}, {"...   
2  [{"name": "Columbia Pictures", "id": 5}, {"nam...   
3  [{"name": "Legendary Pictures", "id": 923}, {"...   
4        [{"name": "Walt Disney Pictures", "id": 2}]   

                                production_countries release_date     revenue  \
0  [{"iso_3166_1": "US", "name": "United States o...   2009-12-10  2787965087   
1  [{"iso_3166_1": "US", "name": "United States o...   2007-05-19   961000000   
2  [{"iso_3166_1": "GB", "name": "United Kingdom"...   2015-10-26   880674609   
3  [{"iso_3166_1": "US", "name": "United States o...   2012-07-16  1084939099   
4  [{"iso_3166_1": "US", "name": "United States o...   2012-03-07   284139100   

   runtime                                   spoken_languages    status  \
0    162.0  [{"iso_639_1": "en", "name": "English"}, {"iso...  Released   
1    169.0           [{"iso_639_1": "en", "name": "English"}]  Released   
2    148.0  [{"iso_639_1": "fr", "name": "Fran\u00e7ais"},...  Released   
3    165.0           [{"iso_639_1": "en", "name": "English"}]  Released   
4    132.0           [{"iso_639_1": "en", "name": "English"}]  Released   

                                          tagline  \
0                     Enter the World of Pandora.   
1  At the end of the world, the adventure begins.   
2                           A Plan No One Escapes   
3                                 The Legend Ends   
4            Lost in our world, found in another.   

                                      title  vote_average  vote_count  
0                                    Avatar           7.2       11800  
1  Pirates of the Caribbean: At World's End           6.9        4500  
2                                   Spectre           6.3        4466  
3                     The Dark Knight Rises           7.6        9106  
4                               John Carter           6.1        2124

In [ ]:
credits.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                                cast  \
0  [{"cast_id": 242, "character": "Jake Sully", "...   
1  [{"cast_id": 4, "character": "Captain Jack Spa...   
2  [{"cast_id": 1, "character": "James Bond", "cr...   
3  [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4  [{"cast_id": 5, "character": "John Carter", "c...   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...  
2  [{"credit_id": "54805967c3a36829b5002c41", "de...  
3  [{"credit_id": "52fe4781c3a36847f81398c3", "de...  
4  [{"credit_id": "52fe479ac3a36847f813eaa3", "de...

In [ ]:
print("Credits:",credits.shape)
print("Movies Dataframe:",movies.shape)

Credits: (4803, 4)
Movies Dataframe: (4803, 20)


In [ ]:
# Rename the credits column to merge on 'id'
credits.columns = ['id', 'title', 'cast', 'crew']
movies_merge = movies.merge(credits, on='id')

In [ ]:
# Drop unnecessary columns
movies_cleaned = movies_merge.drop(columns=['homepage', 'title_x', 'title_y', 'status', 'production_countries'])

In [ ]:
# Parse JSON strings in 'cast', 'crew', and 'genres' columns
features = ['cast', 'crew', 'genres']
for feature in features:
    movies_cleaned[feature] = movies_cleaned[feature].apply(literal_eval)

In [ ]:
# Print info and head to verify
print("Cleaned DataFrame Info:")
print(movies_cleaned.info())
print("\nCleaned DataFrame Head:")
print(movies_cleaned.head())

Cleaned DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   id                    4803 non-null   int64  
 3   keywords              4803 non-null   object 
 4   original_language     4803 non-null   object 
 5   original_title        4803 non-null   object 
 6   overview              4800 non-null   object 
 7   popularity            4803 non-null   float64
 8   production_companies  4803 non-null   object 
 9   release_date          4802 non-null   object 
 10  revenue               4803 non-null   int64  
 11  runtime               4801 non-null   float64
 12  spoken_languages      4803 non-null   object 
 13  tagline               3959 non-null   object 
 14  vote_average          4803 non-null   float64
 1

In [ ]:
# Helper functions to extract names from JSON data
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        # Check if more than 3 elements exist. If yes, return only first three.
        # If no, return the entire list.
        if len(names) > 3:
            names = names[:3]
        return names
    # Return empty list if no data
    return []

# Apply the functions to create new columns
movies_cleaned['crew'] = movies_cleaned['crew'].apply(get_director)
features_to_list = ['cast', 'genres']
for feature in features_to_list:
    movies_cleaned[feature] = movies_cleaned[feature].apply(get_list)

# Function to combine all features into a single string (the "soup")
def create_soup(x):
    # Check for NaN values and convert to empty string to avoid errors
    cast = ' '.join(x['cast']).replace(' ', '') if isinstance(x['cast'], list) else ''
    genres = ' '.join(x['genres']).replace(' ', '') if isinstance(x['genres'], list) else ''
    crew = x['crew'].replace(' ', '') if isinstance(x['crew'], str) and pd.notna(x['crew']) else ''
    overview = x['overview'].replace(' ', '') if isinstance(x['overview'], str) and pd.notna(x['overview']) else ''
    return cast + ' ' + crew + ' ' + genres + ' ' + overview

# Create the new 'soup' column
movies_cleaned['soup'] = movies_cleaned.apply(create_soup, axis=1)

# Initialize TF-IDF Vectorizer on the 'soup'
tfv = TfidfVectorizer(min_df=3, max_features=None,
                      strip_accents='unicode', analyzer='word', token_pattern=r'\w{1,}',
                      ngram_range=(1, 3), stop_words='english')

# Fit and transform the 'soup' to get the TF-IDF matrix
tfv_matrix = tfv.fit_transform(movies_cleaned['soup'])
print("\nTF-IDF Matrix Shape:", tfv_matrix.shape)

# Compute the cosine similarity matrix
cosine_sim = cosine_similarity(tfv_matrix, tfv_matrix)
print("Cosine Similarity Matrix Shape:", cosine_sim.shape)


TF-IDF Matrix Shape: (846, 234)
Cosine Similarity Matrix Shape: (846, 846)


In [ ]:
# Create a reverse mapping of movie titles to their index
indices = pd.Series(movies_cleaned.index, index=movies_cleaned['original_title']).drop_duplicates()

# The recommendation function using the new cosine similarity matrix
def give_recomendations(title, cosine_sim=cosine_sim):
    # Get the index corresponding to the movie title
    if title not in indices:
        return "Movie not found in the dataset. Please check the spelling."
    idx = indices[title]

    # Get the pairwise similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return movies_cleaned['original_title'].iloc[movie_indices]

# Test the recommendation function
print("\nRecommendations for 'The Dark Knight Rises':")
print(give_recomendations('The Dark Knight Rises'))

print("\nRecommendations for 'Avatar':")
print(give_recomendations('Avatar'))


Recommendations for 'The Dark Knight Rises':
119                               Batman Begins
65                              The Dark Knight
95                                 Interstellar
629                              Need for Speed
761                              Righteous Kill
762                              Mercury Rising
96                                    Inception
405       The Fast and the Furious: Tokyo Drift
0                                        Avatar
1      Pirates of the Caribbean: At World's End
Name: original_title, dtype: object

Recommendations for 'Avatar':
20         The Amazing Spider-Man
38       The Amazing Spider-Man 2
107                      Warcraft
126          Thor: The Dark World
627               The Last Legion
786                  西游记之孙悟空三打白骨精
587                     The Abyss
25                        Titanic
282                     True Lies
279    Terminator 2: Judgment Day
Name: original_title, dtype: object


In [ ]:
# Manually inspect recommendations for a few movies
print("Recommendations for 'The Avengers':")
print(give_recomendations('Warcraft'))

print("\nRecommendations for 'The Godfather':")
print(give_recomendations('The Fast and the Furious: Tokyo Drift'))

print("\nRecommendations for 'Pulp Fiction':")
print(give_recomendations('Pulp Fiction'))

Recommendations for 'The Avengers':
38                      The Amazing Spider-Man 2
107                                     Warcraft
126                         Thor: The Dark World
627                              The Last Legion
786                                 西游记之孙悟空三打白骨精
19     The Hobbit: The Battle of the Five Armies
46                    X-Men: Days of Future Past
9             Batman v Superman: Dawn of Justice
14                                  Man of Steel
0                                         Avatar
Name: original_title, dtype: object

Recommendations for 'The Godfather':
629                              Need for Speed
761                              Righteous Kill
56                             Star Trek Beyond
204                                   Fast Five
762                              Mercury Rising
3                         The Dark Knight Rises
119                               Batman Begins
0                                        Avatar
1      Pirates o